In [241]:
using Gadfly 
using DataFrames
using Statsbase

In [259]:
#non-controllable loads
function rand_bldg_loads(n)
    profiles = readtable("stock_res_profiles.csv", header = false)
    df = DataFrames.DataFrame()
    for i in 1:n
        a = profiles[rand(1:5)]*((randn()/4)+1)
        df = hcat(df,a)
    end
    return df
    #writetable("profiles.csv",header=false,df)
end

rand_bldg_loads (generic function with 1 method)

In [2]:
#scheduled loads
function fBuild_Sched(st, fw, nc)
    df = DataFrame()
    for i in 0:(nc-1)
        a = [st,st+fw].+i*sample([36,48,48,60])
        df = hcat(df,a)
    end
    return df
end

fBuild_Sched (generic function with 1 method)

In [299]:
function rand_tech_parameters(n)
    
    PV = randn(n)+5
    B1 = fill(0,n) #pBattNominalE
    B2 = fill(1,n) #pBattDischargeCapacity
    B3 = fill(1,n) #pBattChargeCapacity
    B4 = fill(1,n) #pBatt_DischargeEff
    B5 = fill(1,n) #pBattChargeEff
    B6 = fill(1,n) #pBattCDeg
    B7 = fill(1,n) #pBattInitialSOC
    B8 = fill(1,n) #pBattSOCMax
    B9 = fill(1,n) #pBattSOCMin
    
    H1 = fill(1,n) #pDeadband_HVAC
    H2 = (randn(n)/4)+2 #pCapacitance_HVAC
    H3 = (randn(n)/4)+4 #pResistance_HVAC
    H4 = (randn(n)/4)+2.5 #pCOP_HVAC
    H5 = (randn(n)/4)+5 #pMaxPower_HVAC
    
    W1 = fill(4,n) #pDeadband_WH
    W2 = rand(2:6,n)/10 #pCapacitance_WH
    W3 = rand(100:140,n) #pResistance_WH
    W4 = fill(1,n) #pCOP_WH
    W5 = (randn(n)/4)+4 #pMaxPower_WH
    
    SL1 = sample([7,8,9,17,18,19],n) #pLoadTime
    SL2 = sample([4,5,6,7,8],n) #pFlexWindow
    SL3 = fill(3,n) #pNumCycles
    SL4 = fill(1,n) #pMaxLoad
    SL5 = fill(1,n) #pTotal_SL_kWh
    
    df = hcat(PV,B1,B2,B3,B4,B5,B6,B7,B8,B9,H1,H2,H3,H4,H5,W1,W2,W3,W4,W5,SL1,SL2,SL3,SL4,SL5)
    df = convert(DataFrames.DataFrame, df)
    rename!(df, {:x1=>:PV_Cap,:x2=>:pBattNominalE,:x3=>:pBattDischargeCapacity,:x4=>:pBattChargeCapacity,:x5=>:pBatt_DischargeEff,
                :x6=>:pBattChargeEff,
                :x7=>:pBattCDeg,:x8=>:pBattInitialSOC,:x9=>:pBattSOCMax,:x10=>:pBattSOCMin,:x11=>:pDeadband_HVAC,
                :x12=>:pCapacitance_HVAC,:x13=>:pResistance_HVAC,:x14=>:pCOP_HVAC,:x15=>:pMaxPower_HVAC,:x16=>:pDeadband_WH,
                :x17=>:pCapacitance_WH,:x18=>:pResistance_WH,:x19=>:pCOP_WH,:x20=>:pMaxPower_WH,:x21=>:pLoadTime,
                :x22=>:pFlexWindow, :x23=>:pNumCycles,:x24=>:pMaxLoad,:x25=>:pTotal_SL_kWh,})

    return df
end

rand_tech_parameters (generic function with 1 method)

In [232]:
df = rand_tech_parameters(3)
df

,PV_Cap,pBattNominalE,pBattDischargeCapacity,pBattChargeCapacity,pBatt_DischargeEff,pBattChargeEff,pBattCDeg,pBattInitialSOC,pBattSOCMax,pBattSOCMin,pDeadband_HVAC,pCapacitance_HVAC,pResistance_HVAC,pCOP_HVAC,pMaxPower_HVAC,pDeadband_WH,pCapacitance_WH,pResistance_WH,pCOP_WH,pMaxPower_WH,pLoadTime,pFlexWindow,pNumCycles,pMaxLoad,pTotal_SL_kWh
1,5.9854136544486725,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.1367902832246926,4.117746699103326,2.765115002487954,4.935368129285896,4.0,0.5,137.0,1.0,3.8300854405066,7.0,5.0,3.0,1.0,1.0
2,4.636099682561218,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.360027197329123,4.441986337178566,2.8592990868499544,4.92724546509504,4.0,0.4,135.0,1.0,3.800400532367278,8.0,7.0,3.0,1.0,1.0
3,4.617586800654658,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.006874216159963,3.7532480252493023,2.6540466859736207,4.63432533899021,4.0,0.3,140.0,1.0,4.019032437147287,8.0,7.0,3.0,1.0,1.0


In [251]:
# need function that builds random set of input data
# output = 2 dataframes, which get passed to dr_dre
# running it 1000 times creates 2000 dataframes which can be stored in dfs and looped over

In [258]:
plot(x=randn(100),Geom.histogram(numberbin=100))

LoadError: unrecognized keyword argument "numberbin"
while loading In[258], in expression starting on line 1

In [301]:
df = rand_bldg_loads(3)
temp = df[2]
loads = temp[1:168]

#df = rand_tech_parameters(3)
#temp = df[2,:]
#temp[:PV_Cap]

168-element DataArray{Float64,1}:
 0.680327
 0.620238
 0.603752
 0.601422
 0.668199
 0.938872
 1.30829 
 1.28728 
 1.09491 
 1.09206 
 1.11387 
 1.08964 
 1.05778 
 ⋮       
 1.05778 
 1.05583 
 1.05583 
 1.10862 
 1.16405 
 1.22225 
 1.28337 
 1.28337 
 1.2192  
 1.15824 
 1.10033 
 1.04531 

In [307]:
"outputs"*string(1)*".csv"

"outputs1.csv"